In [5]:
import requests
import pandas as pd
from typing import List, Dict
import time

class DabangRoomCrawler:
    BASE_URL = "https://www.dabangapp.com/api/rooms/list"

    def __init__(self, lat: float, lng: float, zoom: int = 13, room_types: List[str] = None):
        self.lat = lat
        self.lng = lng
        self.zoom = zoom
        self.room_types = room_types or ["onetwo", "apt", "house", "officetel"]

    def fetch_rooms(self, room_type: str) -> List[Dict]:
        params = {
            "m_lat": self.lat,
            "m_lng": self.lng,
            "m_zoom": self.zoom,
            "detail_type": "room",
            "room_type": room_type
        }

        headers = {
            "User-Agent": "Mozilla/5.0",
            "Accept": "application/json"
        }

        try:
            response = requests.get(self.BASE_URL, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()
            return data.get("rooms", [])
        except requests.RequestException as e:
            print(f"[{room_type}] 요청 실패: {e}")
            return []

    def normalize_room(self, room: Dict) -> Dict:
        deposit = room.get("deposit", 0)
        rent = room.get("rent", 0) if room.get("rent_type") == "월세" else None
        maintenance = room.get("maintenance_cost", 0)
        includes = ["관리비 포함"] if room.get("is_maintenance_included") else []

        area_m2 = room.get("size_m2", 0)
        area_py = round(area_m2 / 3.3058, 2)

        return {
            "매물ID": room.get("id"),
            "사이트": "다방",
            "주소": room.get("location", {}).get("address", ""),
            "세부 URL": f"https://www.dabangapp.com/room/{room.get('id')}",
            "유형": room.get("rent_type"),
            "보증금": deposit,
            "월세": rent,
            "관리비": maintenance,
            "관리비 포함": ', '.join(includes),
            "면적(m²)": area_m2,
            "면적(평)": area_py,
            "형식": room.get("room_type"),
            "층수": f"{room.get('floor')}층",
            "등록번호": room.get("id")
        }

    def run(self) -> pd.DataFrame:
        all_rooms = []
        for room_type in self.room_types:
            print(f"수집 중: {room_type}")
            rooms = self.fetch_rooms(room_type)
            all_rooms.extend([self.normalize_room(room) for room in rooms])
            time.sleep(1)  # 서버에 부담 주지 않기
        return pd.DataFrame(all_rooms)

if __name__ == "__main__":
    # 좌표와 줌(반경) 설정
    lat = 37.48066
    lng = 126.89341
    zoom = 13  # 숫자가 클수록 좁은 범위, 11~16 사이 추천

    crawler = DabangRoomCrawler(lat, lng, zoom)
    df = crawler.run()
    df.to_csv("dabang_rooms_result.csv", index=False, encoding="utf-8-sig")
    print("\n✅ 저장 완료: dabang_rooms_result.csv")


수집 중: onetwo
[onetwo] 요청 실패: Expecting value: line 1 column 1 (char 0)
수집 중: apt
[apt] 요청 실패: Expecting value: line 1 column 1 (char 0)
수집 중: house
[house] 요청 실패: Expecting value: line 1 column 1 (char 0)
수집 중: officetel
[officetel] 요청 실패: Expecting value: line 1 column 1 (char 0)

✅ 저장 완료: dabang_rooms_result.csv


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import json
from typing import List, Dict

class DabangElementScraper:
    def __init__(self, lat: float, lng: float, zoom: int = 11, sleep_sec: int = 5):
        self.lat = lat
        self.lng = lng
        self.zoom = zoom
        self.sleep_sec = sleep_sec
        self.url = (
            f"https://www.dabangapp.com/map/onetwo?"
            f"m_lat={self.lat}&m_lng={self.lng}&m_zoom={self.zoom}"
        )

    def start_browser(self):
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        self.driver = webdriver.Chrome(options=options)

    def scrape(self) -> List[Dict]:
        self.start_browser()
        self.driver.get(self.url)
        time.sleep(self.sleep_sec)  # JavaScript 로딩 대기

        results = []

        headers = self.driver.find_elements(By.CSS_SELECTOR, "header.sc-bxRXHk.wDZJC")
        prices = self.driver.find_elements(By.CSS_SELECTOR, "h1.sc-fwdaTD.gTPUHo")

        for header, price_tag in zip(headers, prices):
            try:
                h1_text = header.find_element(By.TAG_NAME, "h1").text.strip()
                if "매물" in h1_text:
                    매물ID = h1_text.replace("매물", "").strip()
                else:
                    매물ID = ""

                가격 = price_tag.text.strip() if price_tag else ""

                results.append({
                    "사이트": "다방",
                    "등록번호": 매물ID,
                    "가격": 가격,
                    "세부 URL": f"https://www.dabangapp.com/room/{매물ID}" if 매물ID else ""
                })
            except Exception as e:
                print("[오류] 요소 파싱 실패:", e)

        self.driver.quit()
        return results

    def save_to_json(self, data: List[Dict], filename: str = "dabang_elements_data.json"):
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"\n✅ JSON 저장 완료: {filename}")


if __name__ == "__main__":
    # 원하는 위도, 경도, 줌(범위) 설정
    scraper = DabangElementScraper(lat=37.4943, lng=127.0144, zoom=11)
    data = scraper.scrape()
    scraper.save_to_json(data)



✅ JSON 저장 완료: dabang_elements_data.json


In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import json
from typing import List, Dict

class DabangElementScraper:
    def __init__(self, lat: float, lng: float, zoom: int = 11, sleep_sec: int = 5):
        self.lat = lat
        self.lng = lng
        self.zoom = zoom
        self.sleep_sec = sleep_sec
        self.url = (
            f"https://www.dabangapp.com/map/onetwo?"
            f"m_lat={self.lat}&m_lng={self.lng}&m_zoom={self.zoom}"
        )

    def start_browser(self):
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')
        self.driver = webdriver.Chrome(options=options)

    def scrape(self) -> List[Dict]:
        self.start_browser()
        self.driver.get(self.url)
        time.sleep(self.sleep_sec)

        results = []

        cards = self.driver.find_elements(By.CSS_SELECTOR, "div[class*='sc-']")

        for card in cards:
            try:
                address = card.find_element(By.CSS_SELECTOR, "p[class*='Address']").text.strip()
                detail_url = card.find_element(By.TAG_NAME, "a").get_attribute("href")
                price_info = card.find_element(By.CSS_SELECTOR, "p[class*='Price']").text.strip()
                type_info = card.find_element(By.CSS_SELECTOR, "p[class*='Type']").text.strip()
                structure_info = card.find_element(By.CSS_SELECTOR, "p[class*='Structure']").text.strip()
                floor_info = card.find_element(By.CSS_SELECTOR, "p[class*='Floor']").text.strip()
                area_info = card.find_element(By.CSS_SELECTOR, "p[class*='Area']").text.strip()
                management_info = card.find_element(By.CSS_SELECTOR, "p[class*='Management']").text.strip()
                listing_id = card.get_attribute("data-id")

                price_parts = price_info.replace(",", "").split('/')
                deposit = price_parts[0].strip()
                rent = price_parts[1].strip() if len(price_parts) > 1 else "0"

                has_management_fee = "없음" not in management_info

                results.append({
                    "사이트": "다방",
                    "매물 주소": address,
                    "세부 매물 정보 URL": detail_url,
                    "유형": type_info,
                    "보증금": deposit,
                    "월세": rent,
                    "관리비": management_info,
                    "관리비 포함 여부": has_management_fee,
                    "면적": area_info,
                    "매물 형식": structure_info,
                    "층수": floor_info,
                    "등록번호": listing_id
                })
            except Exception as e:
                print("[오류] 요소 파싱 실패:", e)

        self.driver.quit()
        return results

    def save_to_json(self, data: List[Dict], filename: str = "dabang_elements_data.json"):
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        print(f"\n✅ JSON 저장 완료: {filename}")


if __name__ == "__main__":
    scraper = DabangElementScraper(lat=37.4943, lng=127.0144, zoom=11)
    data = scraper.scrape()
    scraper.save_to_json(data)


[오류] 요소 파싱 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"p[class*='Address']"}
  (Session info: chrome=135.0.7049.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF723505355+78597]
	GetHandleVerifier [0x00007FF7235053B0+78688]
	(No symbol) [0x00007FF7232B91AA]
	(No symbol) [0x00007FF72330F149]
	(No symbol) [0x00007FF72330F3FC]
	(No symbol) [0x00007FF723301BEC]
	(No symbol) [0x00007FF72333712F]
	(No symbol) [0x00007FF723301AB6]
	(No symbol) [0x00007FF723337300]
	(No symbol) [0x00007FF72335F2BB]
	(No symbol) [0x00007FF723336EC3]
	(No symbol) [0x00007FF7233003F8]
	(No symbol) [0x00007FF723301163]
	GetHandleVerifier [0x00007FF7237AEF0D+2870973]
	GetHandleVerifier [0x00007FF7237A96B8+2848360]
	GetHandleVerifier [0x00007FF7237C6993+2967875]
	GetHandleVerifier [0x00007FF72352019A+188746]
	GetHandleVerifier [

In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 🧭 입력값
LAT = 37.5665
LNG = 126.9780
ZOOM = 17
ROOMTYPES = ['onetwo', 'apt', 'house', 'officetel']

# 🌍 Selenium 설정
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

results = []

# ✅ 유형별 반복
for roomtype in ROOMTYPES:
    url = f"https://www.dabangapp.com/map/{roomtype}?m_lat={LAT}&m_lng={LNG}&m_zoom={ZOOM}"
    print(f"📍 Scanning: {url}")
    driver.get(url)
    time.sleep(3)  # JS 렌더링 기다림

    buttons = driver.find_elements("css selector", "button[aria-label]")
    detail_ids = [btn.get_attribute("aria-label") for btn in buttons if btn.get_attribute("aria-label").isdigit()]

    for detail_id in set(detail_ids):
        detail_url = f"https://www.dabangapp.com/map/{roomtype}?m_lat={LAT}&m_lng={LNG}&m_zoom={ZOOM}&detail_id={detail_id}&detail_type=room"
        print(f"   ↳ Fetching Detail: {detail_id}")

        try:
            response = requests.get(detail_url, headers={"User-Agent": "Mozilla/5.0"})
            soup = BeautifulSoup(response.text, 'html.parser')

            # 🔍 정보 파싱
            text = soup.get_text(separator='\n')

            # 🏷️ 값 추출 함수
            def extract(pattern, text_block):
                import re
                match = re.search(pattern, text_block)
                return match.group(1).strip() if match else ""

            address = extract(r'주소[\s:]*([\w\s\d\-()]+)', text)
            sale_type = extract(r'(매매|전세|월세)', text)
            deposit = extract(r'보증금[\s:]*([\d,]+)', text)
            monthly = extract(r'월세[\s:]*([\d,]+)', text)
            admin_fee = extract(r'관리비[\s:]*([\d,]+)', text)
            admin_included = "포함" in text or "관리비 포함" in text
            size_m2 = extract(r'㎡[\s:]*([\d.]+)', text)
            size_p = extract(r'평[\s:]*([\d.]+)', text)
            floor = extract(r'층[\s:]*([\d]+)', text)
            format_type = roomtype  # 유형 그대로 사용

            results.append({
                "매물ID": detail_id,
                "사이트": "다방",
                "주소": address,
                "세부 URL": detail_url,
                "유형": sale_type,
                "보증금": deposit,
                "월세": monthly,
                "관리비": admin_fee,
                "관리비 포함": admin_included,
                "면적(m²)": size_m2,
                "면적(평)": size_p,
                "형식": format_type,
                "층수": floor
            })
        except Exception as e:
            print(f"❌ Error with detail_id={detail_id}: {e}")

# 종료
driver.quit()

# ✅ DataFrame 저장
df = pd.DataFrame(results)
df.to_csv("dabang_listings.csv", index=False, encoding="utf-8-sig")
print("🎉 저장 완료: dabang_listings.csv")


📍 Scanning: https://www.dabangapp.com/map/onetwo?m_lat=37.5665&m_lng=126.978&m_zoom=15
📍 Scanning: https://www.dabangapp.com/map/apt?m_lat=37.5665&m_lng=126.978&m_zoom=15
📍 Scanning: https://www.dabangapp.com/map/house?m_lat=37.5665&m_lng=126.978&m_zoom=15
📍 Scanning: https://www.dabangapp.com/map/officetel?m_lat=37.5665&m_lng=126.978&m_zoom=15
🎉 저장 완료: dabang_listings.csv
